In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.2 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").getOrCreate()

In [4]:
# Import Amazon Appliance dataset
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [5]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R203HPW78Z7N4K|          5|            0|          0|   N|                Y|
|R2EAIGVLEALSP3|          5|            1|          1|   N|                Y|
|R1K1CD73HHLILA|          5|            0|          0|   N|                Y|
|R2KZBMOFRMYOPO|          5|            1|          1|   N|                Y|
| R6BIZOZY6UD01|          5|            0|          0|   N|                Y|
|R1MCXZFNF8E7Y0|          1|            0|          0|   N|                Y|
|R3EMB3E3ODR6BW|          5|            2|          2|   N|                Y|
| RJTONVTTOPJ5S|          5|            0|          0|   N|                Y|
|R21U5QZ2CQECUM|          4|            0|          0|   N|                Y|
| RL2BBC51H89DH|          4|            0|          0|   N|     

In [8]:
# 1. Filter the data to retrieve all rows where the total_votes >= 20
total_votes_vine_df = vine_df.filter(vine_df.total_votes>=20)
total_votes_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1QXNQBTH7LIUB|          3|            9|         22|   N|                Y|
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|
|R16LPVYSJOAH1T|          1|           20|         20|   N|                N|
|R2NLL7ISMM5QYQ|          4|           89|         94|   N|     

In [9]:
# 2. Create new df to retrieve all rows where the number of helpful votes divided by total votes >= 50%
percentage_vine_df = total_votes_vine_df.filter(total_votes_vine_df.helpful_votes / total_votes_vine_df.total_votes >= 0.5)
percentage_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|
|R16LPVYSJOAH1T|          1|           20|         20|   N|                N|
|R2NLL7ISMM5QYQ|          4|           89|         94|   N|                Y|
|R3FAFI6Q0YL37W|          1|           30|         35|   N|     

In [12]:
# 3. Filter by paid reviews
paid_filtered_vine_df = percentage_vine_df.filter(percentage_vine_df.vine == 'Y')
paid_filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R20O97P5V9CZ21|          3|           21|         27|   Y|                N|
| RCNB7YGLWKCV5|          5|           23|         30|   Y|                N|
| RX1ZUVGBH3HKC|          5|          814|        848|   Y|                N|
| RRS5AGL10DYU3|          3|           25|         29|   Y|                N|
| R5YFC5G8V2B3I|          5|          104|        117|   Y|                N|
|R1IDQ0G1S7J9TN|          5|          178|        196|   Y|                N|
| RKXNZX22HQZEE|          3|          182|        192|   Y|                N|
|R2K45IY0NO70ZI|          5|          116|        122|   Y|                N|
| RQXRI4UKDX877|          5|           20|         24|   Y|                N|
|R1L2R8OHO171YG|          5|           66|         68|   Y|     

In [13]:
# 4. Filter by unpaid reviews
unpaid_filtered_vine_df = percentage_vine_df.filter(percentage_vine_df.vine == 'N')
unpaid_filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|
|R16LPVYSJOAH1T|          1|           20|         20|   N|                N|
|R2NLL7ISMM5QYQ|          4|           89|         94|   N|                Y|
|R3FAFI6Q0YL37W|          1|           30|         35|   N|     

In [21]:
# 5a. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for paid reviews
total_reviews_paid = paid_filtered_vine_df.count()
five_star_reviews_paid = paid_filtered_vine_df.filter(paid_filtered_vine_df.star_rating == 5).count()
per_five_star_reviews_paid = five_star_reviews_paid / total_reviews_paid * 100

print("Paid Review Statistics")
print("----------------------")
print(f"Total Reviews: {total_reviews_paid}\nTotal Five Star Revews: {five_star_reviews_paid}\nPercentage of Five Star Reviews: {'{:.2f}'.format(per_five_star_reviews_paid)}%")

Paid Review Statistics
----------------------
Total Reviews: 35
Total Five Star Revews: 18
Percentage of Five Star Reviews: 51.43%


In [22]:
# 5b. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for unpaid reviews
total_reviews_unpaid = unpaid_filtered_vine_df.count()
five_star_reviews_unpaid = unpaid_filtered_vine_df.filter(unpaid_filtered_vine_df.star_rating == 5).count()
per_five_star_reviews_unpaid = five_star_reviews_unpaid / total_reviews_unpaid * 100

print("Unpaid Review Statistics")
print("----------------------")
print(f"Total Reviews: {total_reviews_unpaid}\nTotal Five Star Revews: {five_star_reviews_unpaid}\nPercentage of Five Star Reviews: {'{:.2f}'.format(per_five_star_reviews_unpaid)}%")

Unpaid Review Statistics
----------------------
Total Reviews: 4957
Total Five Star Revews: 1963
Percentage of Five Star Reviews: 39.60%
